In [2]:
# -*- coding:utf-8 -*-
#导入相关包
import os,sys
import time
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
#设置路径,读取数据
train_savepath = 'D:/worksp/jupyter/Model/data/train/hbsubs/'
test_savepath = 'D:/worksp/jupyter/Model/data/test/hbsubs/'
model_savepath = 'D:/worksp/jupyter/Model/Model_path/hbsubs/'
result_path = 'D:/worksp/jupyter/Model/result_path/'
df = pd.read_csv(train_savepath + 'lgb_train_allprod_2000k.csv')

In [2]:
#label转换
df['label'].loc[df['label']=='a'] = 0
df['label'].loc[df['label']=='B232080'] = 1
df['label'].loc[df['label']=='B232081'] = 1
df['label'].loc[df['label']=='B232082'] = 1
df['label'].loc[df['label']=='B232083'] = 1
df['label'].loc[df['label']=='B232084'] = 1
df['label'].loc[df['label']=='B232085'] = 1
#对连续型数值字段构建新特征
def numcol_engin():
    numcol = ['l2499','l2277','l2447','l2457','l3709','l2269','l2271','l3707','l2335','l2337']
    for i in numcol:
        mean = df[i].mean()
        std = df[i].std()
        df[i+'e'] = df[i].apply(lambda x: abs((x - mean ) / (std)))
#部分字段特征工程
df['l2373'] = df['l2373']+df['l2375']+df['l2377']+df['l2379']+df['l2381']+df['l2383']
df['l2373'].loc[df['l2373']>0] = 1
df['l2097'].loc[df['l2097']>2019] = 0
#df['l2271e2'] = df['l2271'] - 78
df['l2447l3709e'] = df['l2447'] - df['l3709']
numcol_engin()

d:\program files (x86)\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [3]:
#按50%、30%、20%划分下层模型训练集、上层模型训练集、验证集
df1 = df.iloc[0:int(df.shape[0]*0.5)+1,:]
df2 = df.iloc[int(df.shape[0]*0.5)+1:int(df.shape[0]*0.8)+1,:]
df3 = df.iloc[int(df.shape[0]*0.8)+1:int(df.shape[0])+1,:]
#设置不同数据集的特征和label
model1_feature = ['l2271','l2499','l2269','l2457','l3709','l2277','l2447','l2555','l2119','l3951','l3707',
                  'l2499e','l2277e','l2447e','l2457e','l3709e','l2269e','l2271e','l3707e','l2447l3709e']
model2_feature = ['l2117','l2257','l7265','l2373','l1309','l1383','l1431','l2105','l2335','l2337','l2517'
                 ,'l2599','l2935','l2337e','l2335e']
model3_feature = ['l3687','l2113','l2109','l2121','l1705','l2097']

model1_train1 = df1.loc[:,model1_feature]
model2_train1 = df1.loc[:,model2_feature]
model3_train1 = df1.loc[:,model3_feature]
y_train1 = df1.loc[:,['label']]
#底层模型的lgb数据格式训练集
lgb_model1_train1 = lgb.Dataset(model1_train1, y_train1, silent=True)
lgb_model2_train1 = lgb.Dataset(model2_train1, y_train1, silent=True)
lgb_model3_train1 = lgb.Dataset(model3_train1, y_train1, silent=True)

In [15]:
#设置训练集列表
df_list = [lgb_model1_train1,lgb_model2_train1,lgb_model3_train1]
#设置CV参数
cv_params = {'boosting_type': 'gbdt', 
    'objective': 'binary', 
    'learning_rate': 0.1, 
    'num_leaves': 50, 
    'max_depth': 5,    
    'subsample': 0.8,
    #'scale_pos_weight':scale_pos_weight[0]
    #'colsample_bytree': 0.8
    }
#求弱分类器数目
best_n_estimators_list = []
best_cv_score = []
def n_estimators_fun(df_list):
    global best_n_estimators_list
    global best_cv_score
    for i in df_list:
        cv_results = lgb.cv(
        cv_params, i, num_boost_round=2000, nfold=5, stratified=True, shuffle=True, metrics='auc',
        early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)
        best_n_estimators_list.append(len(cv_results['auc-mean']))
        best_cv_score.append(cv_results['auc-mean'][-1])

In [16]:
begintime=time.clock()
n_estimators_fun(df_list)
endtime=time.clock()
print (endtime-begintime)

[50]	cv_agg's auc: 0.957041 + 0.000666212
[100]	cv_agg's auc: 0.960901 + 0.000951849
[150]	cv_agg's auc: 0.962247 + 0.000956301
[200]	cv_agg's auc: 0.963508 + 0.000813061
[250]	cv_agg's auc: 0.964423 + 0.000627293
[300]	cv_agg's auc: 0.964733 + 0.000502314
[350]	cv_agg's auc: 0.96489 + 0.000470186
[400]	cv_agg's auc: 0.965094 + 0.000413456
[50]	cv_agg's auc: 0.819872 + 0.00315195
[100]	cv_agg's auc: 0.821582 + 0.00325034
[150]	cv_agg's auc: 0.82224 + 0.00308538
[50]	cv_agg's auc: 0.780827 + 0.0015575
[100]	cv_agg's auc: 0.780644 + 0.00132346
186.52865009999994


In [36]:
mcw=1/pow(y_train1.sum()[0],0.5)
model_lgb = lgb.LGBMClassifier(objective='binary'
                            ,num_leaves=50
                            ,learning_rate=0.1
                            ,n_estimators=best_n_estimators_list[1]
                            ,max_depth=6
                            ,metric='binary_logloss'
                            ,bagging_fraction = 0.8
                            ,bagging_freq = 5
                            ,n_jobs=8
                            #,feature_fraction = 0.8
                            )
params_test1={
    'n_estimators':[best_n_estimators_list[2]]
    ,'learning_rate':np.arange(0.01,0.1,0.01)
    ,'min_child_weight':[mcw]
    ,'min_child_samples': [18, 19, 20, 21, 22]
    ,'max_depth': range(3,8,1)
    ,'num_leaves':range(8, 248, 20)
    ,'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0]
    ,'bagging_freq':[5]
    ,'reg_alpha':np.arange(0,4,0.5)
    ,'reg_lambda':np.arange(0,4,0.5)
}
clf = RandomizedSearchCV(estimator=model_lgb, 
                              param_distributions=params_test1, 
                              scoring='roc_auc', 
                              cv=5,
                              n_iter=10,
                              verbose=1
                              )
begintime=time.clock()
clf.fit(model3_train1, y_train1)
endtime=time.clock()
print (endtime-begintime)
clf.score, clf.best_params_, clf.best_score_#根据设置的测试参数，计算出最优模型参数

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: Data

d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\program files (x86)\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

224.86769139999979


(<bound method BaseSearchCV.score of RandomizedSearchCV(cv=5, error_score='raise-deprecating',
           estimator=LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
         class_weight=None, colsample_bytree=1.0, importance_type='split',
         learning_rate=0.1, max_depth=6, metric='binary_logloss',
         min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
   ...0.0, reg_lambda=0.0, silent=True,
         subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
           fit_params=None, iid='warn', n_iter=10, n_jobs=None,
           param_distributions={'n_estimators': [59], 'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]), 'min_child_weight': [0.01060414979163009], 'min_child_samples': [18, 19, 20, 21, 22], 'max_depth': range(3, 8), 'num_leaves': range(8, 248, 20), 'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0], 'bagging_freq': [5], 'reg_alpha': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5]), 'reg_lambda

In [37]:
lgb_model3_param = clf.best_params_
savemodel=lgb.train(lgb_model3_param,lgb_model3_train1)
savemodel.save_model(model_savepath+'lgb_model3_blending')

d:\program files (x86)\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [3]:
bst1 = lgb.Booster(model_file=model_savepath+'lgb_model1_blending')
bst2 = lgb.Booster(model_file=model_savepath+'lgb_model2_blending')
bst3 = lgb.Booster(model_file=model_savepath+'lgb_model3_blending')

In [5]:
#设置上层模型训练集
model1_test2 = df2.loc[:,model1_feature]
model2_test2 = df2.loc[:,model2_feature]
model3_test2 = df2.loc[:,model3_feature]

In [6]:
model1_pre = bst1.predict(model1_test2, num_iteration=bst1.best_iteration)
model2_pre = bst2.predict(model2_test2, num_iteration=bst2.best_iteration)
model3_pre = bst3.predict(model3_test2, num_iteration=bst3.best_iteration)

In [7]:
model_upper_df = pd.DataFrame(df2['label'])
model_upper_df['feature1'] = model1_pre
model_upper_df['feature2'] = model2_pre
model_upper_df['feature3'] = model3_pre
model_upper_feature = ['feature1','feature2','feature3']
model_upper_train = model_upper_df.loc[:,model_upper_feature]
model_upper_label = model_upper_df['label']

In [8]:
lgb_model_upper_train = lgb.Dataset(model_upper_train, model_upper_label, silent=True)

In [89]:
cv_results = lgb.cv(
        cv_params,lgb_model_upper_train, num_boost_round=2000, nfold=5, stratified=True, shuffle=True, metrics='auc',
        early_stopping_rounds=50, verbose_eval=50, show_stdv=True, seed=0)
print('best_n_estimators:', len(cv_results['auc-mean']))
print('best_cv_score:', cv_results['auc-mean'][-1])
model_upper = lgb.LGBMClassifier(objective='binary'
                            ,num_leaves=50
                            ,learning_rate=0.1
                            ,n_estimators=len(cv_results['auc-mean'])
                            ,max_depth=6
                            ,metric='binary_logloss'
                            ,bagging_fraction = 0.8
                            ,bagging_freq = 5
                            ,n_jobs=8
                            #,feature_fraction = 0.8
                            )
params_test1={
    'n_estimators':[len(cv_results['auc-mean'])]
    ,'learning_rate':np.arange(0.01,0.1,0.01)
    ,'min_child_weight':[mcw]
    ,'min_child_samples': [18, 19, 20, 21, 22]
    ,'max_depth': range(3,8,1)
    ,'num_leaves':range(8, 248, 20)
    ,'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0]
    ,'bagging_freq':[5]
    ,'reg_alpha':np.arange(0,4,0.5)
    ,'reg_lambda':np.arange(0,4,0.5)
}
clf = RandomizedSearchCV(estimator=model_lgb, 
                              param_distributions=params_test1, 
                              scoring='roc_auc', 
                              cv=5,
                              n_iter=10,
                              verbose=1
                              )
begintime=time.clock()
clf.fit(model_upper_train,model_upper_label)
endtime=time.clock()
print (endtime-begintime)
clf.score, clf.best_params_, clf.best_score_#根据设置的测试参数，计算出最优模型参数

[50]	cv_agg's auc: 0.901235 + 0.0369728
best_n_estimators: 1
best_cv_score: 0.9488706686763493
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   31.5s finished


32.0169292999999


(<bound method BaseSearchCV.score of RandomizedSearchCV(cv=5, error_score='raise-deprecating',
           estimator=LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
         class_weight=None, colsample_bytree=1.0, importance_type='split',
         learning_rate=0.1, max_depth=6, metric='binary_logloss',
         min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
   ...0.0, reg_lambda=0.0, silent=True,
         subsample=1.0, subsample_for_bin=200000, subsample_freq=0),
           fit_params=None, iid='warn', n_iter=10, n_jobs=None,
           param_distributions={'n_estimators': [1], 'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]), 'min_child_weight': [0.01060414979163009], 'min_child_samples': [18, 19, 20, 21, 22], 'max_depth': range(3, 8), 'num_leaves': range(8, 248, 20), 'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0], 'bagging_freq': [5], 'reg_alpha': array([0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5]), 'reg_lambda'

In [7]:
upper_model1_param = clf.best_params_
savemodel=lgb.train(upper_model1_param,lgb_model_upper_train)
savemodel.save_model(model_savepath+'lgb_model_upper_blending')

NameError: name 'clf' is not defined

In [9]:
bst_upper = lgb.Booster(model_file=model_savepath+'lgb_model_upper_blending')

In [10]:
#设置验证集
model1_test3 = df3.loc[:,model1_feature]
model2_test3 = df3.loc[:,model2_feature]
model3_test3 = df3.loc[:,model3_feature]
model1_pre2 = bst1.predict(model1_test3, num_iteration=bst1.best_iteration)
model2_pre2 = bst2.predict(model2_test3, num_iteration=bst2.best_iteration)
model3_pre2 = bst3.predict(model3_test3, num_iteration=bst3.best_iteration)

In [11]:
model_upper_df1 = pd.DataFrame()
model_upper_df1['feature1'] = model1_pre2
model_upper_df1['feature2'] = model2_pre2
model_upper_df1['feature3'] = model3_pre2
model_upper_feature = ['feature1','feature2','feature3']
model_upper_test = model_upper_df1.loc[:,model_upper_feature]

In [12]:
upper_model_pre = bst_upper.predict(model_upper_test, num_iteration=bst3.best_iteration)

In [13]:
df_result = pd.DataFrame(upper_model_pre)
estimator = KMeans(n_clusters=10)
estimator.fit(df_result)
label_pred = estimator.labels_ #获取聚类标签
centroids = estimator.cluster_centers_ #获取聚类中心
df_result['Kmeans'] = label_pred
df_result1 = pd.DataFrame(df_result)
#根据预测概率，从高到低改变聚类标签
for i in range(10):
    df_result1['Kmeans'].ix[df_result1['Kmeans'] ==i]=str(sorted(centroids).index(centroids[i]))+"星"

d:\program files (x86)\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.
d:\program files (x86)\anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
#df3['label'].to_csv('./blending_test.csv')
result_file_path = result_path+"result_blending_4.xlsx"
df_result1.to_excel(result_file_path)